# Práctica 4

**Nombre:** Javier Rosales Martínez     
**e-mail:** j.rosales@alumnos.udg.mx

## MODULES

In [1]:
import panel as pn
import panel.widgets as pnw
pn.extension('plotly')

import pandas as pd
import numpy as np

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

import math

# Classes

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# Functions

In [72]:
rw_ = []
def bm_2d(n_steps = 1000, speed = 6, s_pos = 0):
    """
    Arguments:
        n_steps:
        speed:
        s_pos:
    Return:
        BM_2d_df
    """
    # Init velocity vector
    velocity = Vec2d(speed, 0)

    # Matriz para Brownian Walker
    BM_2d = np.ones(shape = (n_steps, 2))*s_pos
    rw_.clear()

    for i in range(1, n_steps):
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)

        BM_2d[i, 0] = BM_2d[i-1, 0]+velocity.x
        BM_2d[i, 1] = BM_2d[i-1, 1]+velocity.y
        rw_.append(BM_2d[i])

def crw_2d(n_steps = 1000, speed = 6, s_pos = 0, QRW_exponent = 0.5):
    """
    Arguments:
        n_steps:
        speed:
        s_pos:
        QRW_exponent:
    Return:
        CRW_2d_df
    """
    # Generate vector of wrapcauchy values
    wrapcauchy_rvs = wrapcauchy.rvs(QRW_exponent, size = n_steps)
    # Init velocity vector
    velocity = Vec2d(speed, 0)

    # Matriz para Brownian Walker
    CRW_2d = np.ones(shape = (n_steps, 2))*s_pos
    rw_.clear()

    for i in range(1, n_steps):
        velocity = velocity.rotated(wrapcauchy_rvs[i])

        CRW_2d[i, 0] = CRW_2d[i-1, 0]+velocity.x
        CRW_2d[i, 1] = CRW_2d[i-1, 1]+velocity.y
        rw_.append(CRW_2d[i])

def Levy_2d(n_steps = 1000, speed = 1, s_pos = 0, QRW_exponent = 0.5,  alpha = 1):
    """
    Arguments:
        n_steps:
        speed:
        s_pos:
        QRW_exponent:
        alpha
    Return:
        CRW_2d
    """
    # Init parameters
    beta = 1
    stdMotionSteps = 6

    # Init velocity vector
    velocity = Vec2d(speed, 0)

    # Init DF
    LW_2d = np.ones(shape = (n_steps, 2))*s_pos
    rw_.clear()

    i = 1
    while i < n_steps-1:
        # Get ramdom n_steps from Levy distribution
        step_size = levy_stable.rvs(alpha, beta, stdMotionSteps)

        # Round to integer number
        step_size = int(np.ceil(abs(step_size)))

        theta = wrapcauchy.rvs(c=0.7, loc=0)

        # Uptade velocity
        velocity = velocity.rotated(theta)

        for j in range(step_size):
            if(i>=n_steps):
                break
            else:
                LW_2d[i, 0] = LW_2d[i-1, 0]+velocity.x
                LW_2d[i, 1] = LW_2d[i-1, 1]+velocity.y
            rw_.append(LW_2d[i])
            # Add to the end to Levy's DF
            i += 1

def path_length(rw = np.ones(shape = (10000, 2))*0):
    dis_rw = np.array([distance.euclidean(rw[i-1], rw[i]) for i in range(1,len(rw))])
    pl_rw = np.cumsum(dis_rw)
    return(pl_rw)


def MSD(rw = np.ones(shape = (10000, 2))*0):
    # Empty MSD_BM
    MSD_BM = np.empty(shape=(0))
    MSD_BM_list = []
    # MSD for BM_2d_df_6
    for tau in range(1,len(rw)):
        ## start - Add your code here
        displacement_vec_bm = np.array([distance.euclidean(rw[i-tau], rw[i]) 
                      for i in range(tau, len(rw),1)])
        MSD_BM_list.append(np.sum(np.power(displacement_vec_bm, 2))/len(displacement_vec_bm))
        ## end - Add your code here
    MSD_BM = np.asarray(MSD_BM_list)
    return(MSD_BM)

def plot_trajec(rw = np.ones(shape = (10000, 2))*0, n_steps=1000):
    times = np.linspace(0,1, n_steps)
    fig_traj_rw = go.Figure()
    
    fig_traj_rw.add_trace(go.Scatter3d(
        x = rw[:,0],
        y = rw[:,1],
        z = times,
        marker = dict(size=2),
        line = dict(color='red', width=2),
        mode = 'lines',
        name = f'steps = {n_steps}',
        showlegend = True
    ))
        
    return fig_traj_rw

def plot_metrics_(pl = np.ones(shape = (10000, 2))*0):
    fig_metrics_rw = go.Figure()
    
    fig_metrics_rw.add_trace(go.Scatter(
        x = np.arange(len(pl)),
        y = pl,
        marker = dict(size=2),
        line = dict(width=2),
        mode = 'lines',
        name = 'path_length_BM_3',
        showlegend = True
    ))
    return fig_metrics_rw

# Trajectory

In [82]:
# Widgets
n_steps = pnw.IntSlider(name='Number of steps', width = 380, value=10, step=10, start=10, end=10000)
s_x_pos = pnw.IntInput(name='Starting pos X', value=10, step=10, start=-100, end=100)
QRW_exponent = pnw.FloatSlider(name='QRW Exponent', width=380, start=0.1, end=0.9)
alpha = pnw.FloatSlider(name='Alpha', width=380, value=1.0, start=0.1, end=1.0)
metric = pn.widgets.Select(name='Metric', options={'PL': 0, 'MSD': 1})

# Plot grafics
@pn.depends(n_steps, s_x_pos)
def plot_traj_bm(n_steps, s_x_pos):
    bm_2d(n_steps, s_x_pos)
    bm_2d_ = np.asarray(rw_)
    return plot_trajec(bm_2d_, n_steps)

@pn.depends(n_steps, s_x_pos, QRW_exponent)
def plot_traj_crw(n_steps, s_x_pos, QRW_exponent):
    crw_2d(n_steps, 5, s_x_pos, QRW_exponent)
    crw_2d_ = np.asarray(rw_)
    return plot_trajec(crw_2d_, n_steps)

@pn.depends(n_steps, s_x_pos, QRW_exponent, alpha)
def plot_traj_lf(n_steps, s_x_pos, QRW_exponent, alpha):
    Levy_2d(n_steps, 5, s_x_pos, QRW_exponent, alpha)
    Levy_2d_ = np.asarray(rw_)
    return plot_trajec(Levy_2d_, n_steps)

# Plot metrics
@pn.depends(n_steps, s_x_pos, metric)
def plot_metrics_bm(n_steps, s_x_pos, metric):
    if(metric == 0):
        rw = np.asarray(rw_)
        pl = path_length(rw)
    elif(metric == 1):
        rw = np.asarray(rw_)
        pl = MSD(rw)
    return plot_metrics_(pl)

@pn.depends(n_steps, s_x_pos, QRW_exponent, metric)
def plot_metrics_crw(n_steps, s_x_pos, QRW_exponent, metric):
    if(metric == 0):
        rw = np.asarray(rw_)
        pl = path_length(rw)
    elif(metric == 1):
        rw = np.asarray(rw_)
        pl = MSD(rw)
    return plot_metrics_(pl)

@pn.depends(n_steps, s_x_pos, QRW_exponent, alpha, metric)
def plot_metrics_lf(n_steps, s_x_pos, QRW_exponent, alpha, metric):
    if(metric == 0):
        rw = np.asarray(rw_)
        pl = path_length(rw)
    elif(metric == 1):
        rw = np.asarray(rw_)
        pl = MSD(rw)
    return plot_metrics_(pl)

# View grafics
p_bm = pn.Column(
    pn.Row('Panel params'),
    pn.Row(n_steps, s_x_pos, metric),
    pn.Row('Grafics'),
    pn.Row(plot_traj_bm,plot_metrics_bm)
)

p_crw = pn.Column(
    pn.Row('Panel params'),
    pn.Row(n_steps, s_x_pos, metric, QRW_exponent),
    pn.Row('Grafics'),
    pn.Row(plot_traj_crw,plot_metrics_crw)
)

p_lf = pn.Column(
    pn.Row('Panel params'),
    pn.Row(n_steps, s_x_pos, metric, QRW_exponent, alpha),
    pn.Row(alpha),
    pn.Row('Grafics'),
    pn.Row(plot_traj_lf,plot_metrics_lf)
)

tabs = pn.Tabs()

tabs.extend([
    ('BM', p_bm),
    ('CRW', p_crw),
    ('Levy', p_lf)
])

tabs

Tabs
    [0] Column
        [0] Row
            [0] Markdown(str)
        [1] Row
            [0] IntSlider(end=10000, name='Number of steps', start=10, step=10, value=10, width=380)
            [1] IntInput(end=100, name='Starting pos X', start=-100, step=10, value=10)
            [2] Select(name='Metric', options={'PL': 0, 'MSD': 1}, value=0)
        [2] Row
            [0] Markdown(str)
        [3] Row
            [0] ParamFunction(function)
            [1] ParamFunction(function)
    [1] Column
        [0] Row
            [0] Markdown(str)
        [1] Row
            [0] IntSlider(end=10000, name='Number of steps', start=10, step=10, value=10, width=380)
            [1] IntInput(end=100, name='Starting pos X', start=-100, step=10, value=10)
            [2] FloatSlider(end=0.9, name='QRW Exponent', start=0.1, value=0.1, width=380)
            [3] Select(name='Metric', options={'PL': 0, 'MSD': 1}, value=0)
        [2] Row
            [0] Markdown(str)
        [3] Row
            [0] ParamFunction(function)
            [1] ParamFunction(function)
    [2] Column
        [0] Row
            [0] Markdown(str)
        [1] Row
            [0] IntSlider(end=10000, name='Number of steps', start=10, step=10, value=10, width=380)
            [1] IntInput(end=100, name='Starting pos X', start=-100, step=10, value=10)
            [2] Select(name='Metric', options={'PL': 0, 'MSD': 1}, value=0)
            [3] FloatSlider(end=0.9, name='QRW Exponent', start=0.1, value=0.1, width=380)
            [4] FloatSlider(name='Alpha', start=0.1, value=1.0, width=380)
        [2] Row
            [0] FloatSlider(name='Alpha', start=0.1, value=1.0, width=380)
        [3] Row
            [0] Markdown(str)
        [4] Row
            [0] ParamFunction(function)
            [1] ParamFunction(function)